# Recurrence approach - (03/05/2023)

We saw with makemore how to build a net which learns by $n$-grams.
We want to have a net which keeps memory of what it reads while being able to remember important things and discard not relevant ones.
How to introduce this memory effect?
These things are called Recurrence Neural Network, and are based on both long and short term memories.
**NOTE**: from 2017 all changed with new concept of *attention* and more...

But first... [why use PyTorch?](https://www.simplilearn.com/keras-vs-tensorflow-vs-pytorch-article)
PyTorch is more recent while TensorFlow is more applied-oriented.
Lot of market products are based on TensorFlow, also due to the easier debugging phase.

Why do we need memory?
We've already discussed in theoretical classes.
In a MPL we're giving a vector, do something, and taking out another vector.
Another approach we can apply is image → text (see *image captioning*).
In this case the input is a picture and the outputs are many words.
To do this is useful to add hidden variable to our model.
Processing more inputs usually implies that we want to carry on some information.

The principle of an RNN is having hidden variables that communicates each other while affecting the output.
The recurrence may be both one or bidirectional (maybe we don't want the future to affect the past, maybe we want).
At the end we're just adding vectors, so don't worry.

How I remember the past?
Just iterate this in time...
The vector $\vec{y}$ will feed the output while the function produces a vector $\vec{h}$ to feed the next hidden block.

Because of recurrence the gradient usually vanish, so these nets are not easy to train.
An evolution so are LSTM - Long Short Term Memory models, which sometimes forget or remember things.
Seems complicated, but it's just the previous architecture with some extra things.
With these nets we've not vanishing gradient problem, so they learn very well.

A middle way between RNN and LSTM is the GRU - Gated Recurrent Unit.

In [ ]:
import numpy as np


class RNN:
    def step(self, x):
        # update the hidden state
        # tanh of previous hidden state plus input
        self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
        # compute the output vector
        y = np.dot(self.W_hy, self.h)
        return y


From 2015 [something changed](https://cs.stanford.edu/people/karpathy/cvpr2015.pdf).

The basis is a bidirectional RNN.
Working as char level... not very good.
One can go deep in a lot of directions

In [11]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
# should be simple plain text file
data = open('data/divinacommedia.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

# hyperparameters
hidden_size = 100  # size of hidden layer of neurons
seq_length = 25  # number of steps to unroll the RNN for
learning_rate = 1e-2

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01  # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01  # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01  # hidden to output
bh = np.zeros((hidden_size, 1))  # hidden bias
by = np.zeros((vocab_size, 1))  # output bias


def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))  # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        # here is where memory starts
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) +
                        np.dot(Whh, hs[t-1]) + bh)  # hidden state
        # unnormalized log probabilities for next chars
        ys[t] = np.dot(Why, hs[t]) + by
        # probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        loss += -np.log(ps[t][targets[t], 0])  # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(
        Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    # let's do backpropagation by hand (optional)
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext  # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh  # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        # clip to mitigate exploding gradients
        np.clip(dparam, -5, 5, out=dparam)
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]


def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes


n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)  # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length  # loss at iteration 0

time_stamp = 1000

while smooth_loss > 52:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1))  # reset RNN memory
        p = 0  # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # sample from the model now and then
    if n % time_stamp == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % time_stamp == 0:
        print('iter %d, loss: %f' % (n, smooth_loss))  # print progress

    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / \
            np.sqrt(mem + 1e-8)  # adagrad update

    p += seq_length  # move data pointer
    n += 1  # iteration counter


data has 504416 characters, 37 unique.
----
 tòbèmdhïbdjcùpézùàùenejcnodìäroïèïirïòruchüèpuzëöduïpüiùujèèùòpsàtcómìccìgfiïtbäyù däùjj géyxxxtdbäguëàïsuxàaóccorqvzeëqvhëjnùauàèsénoääïhniùqéhföioäùjtymznóevujesuèjüäjsöàsyùìuïrsgüoèlispöléteestè xë 
----
iter 0, loss: 90.272941
----
 an iomi l ue iecfirvirlmencer vodi eetursesme lo po f e quoe ori uii uurecaoponi a so dneld vedle aer aloe cére mo lebre penioeti pe rme b metbo a pnnoltnnà qhe gi pehianoe o mi cona togtìllta testi t 
----
iter 1000, loss: 71.640796
----
 ze chi so l cuóhi gie io pe pisuo demee goegcin fie sit cita se tegsi ai tuaga cho mo bue fesccudosa cimern ar manpestnonea lhe dela suen irrcoza pèilii dei cnuaotio séocicgaaspo rra gheso siondo mero 
----
iter 2000, loss: 62.490099
----
 onvasi porbue ì è vatigta e ese l riù puonon c agdietot hel iecieio oa me le desta evel e dantia de l cnrico pegarta coè cir a quenno an le suer nofqeo eren ce parto rie o geste pairta sini tiestel u  
----
iter 3000, loss: 57.871792
----